In [1]:
import psi4
import fortecubeview as vis
import glob

## Vibrational frequency analysis can be used to verify a transition state structure.
## Here, we have a rough guess of a transition state structure of CNH -> HCN reaction.

In [2]:
CNH  = """
      0 1 
      C  0.0  0.0  0.0 
      N  0.0  1.16  0.0
      H  0.0  0.6  1.0
      """
vis.geom(xyz=CNH)

Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

## We will optimize it first and obtain vibrational modes.

In [3]:
M = psi4.geometry(CNH)
psi4.core.set_output_file('TSoptimization.out')
psi4.set_options({'opt_type':'TS'})
energy, wfn = psi4.optimize('hf/6-31g', molecule = M,return_wfn = True)
print('Energy of the optimized water is %.7f' %energy)
vis.geom(molecule = M)

Optimizer: Optimization complete!
Energy of the optimized water is -92.7236056


Renderer(camera=OrthographicCamera(bottom=-5.0, children=(DirectionalLight(color='white', intensity=0.5, posit…

In [4]:
psi4.set_options({"normal_modes_write ":True,
                  "writer_file_label":"CNH"})
psi4.core.set_output_file('vib.out')
E, wfn = psi4.frequency('hf/6-31g', molecule=M, return_wfn=True)
print(wfn.frequencies().np[:])

[-1246.0237264   2137.59839405  2548.36417618]


## One imaginary frequency confirms that the molecule is located at a saddle point.
## Visualized vector of that imaginary frequency shows a direction to one of the two energy minima.

In [5]:
vis.vib(glob.glob('CNH.*.molden_normal_modes')[0])

HTML(value='\n        <style>\n           .jupyter-widgets-output-area .output_scroll {\n                heigh…

interactive(children=(Select(description='Select:', options=('1: Imaginary mode (i1246.0 cm^-1)', '2: Normal m…